# Greedy Algorithms

*Making locally optimal choices to find global solutions*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_13.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand the greedy paradigm: making locally optimal choices that lead to globally optimal solutions
- Distinguish when greedy works vs. when DP is needed, using the greedy choice property and optimal substructure
- Solve interval scheduling and interval partitioning problems using greedy sorting strategies
- Implement Huffman coding, Dijkstra's shortest path, and minimum spanning tree algorithms (Kruskal, Prim)
- Recognize common greedy patterns and prove (or disprove) greedy correctness via exchange arguments

---
## 1. Introduction to Greedy Algorithms


Greedy algorithms build solutions piece by piece, always choosing the option that looks best at the moment. Unlike dynamic programming which considers all possibilities, greedy makes an irrevocable decision at each step. When applicable, greedy algorithms are typically simpler and more efficient than DP.

> 📖 **Definition:** A **greedy algorithm** makes the locally optimal choice at each step, hoping this leads to a globally optimal solution. For greedy to work, the problem must exhibit **greedy choice property** (local optimal leads to global optimal) and **optimal substructure** (optimal solution contains optimal solutions to subproblems).

### When Does Greedy Work?

| Property | Description | Example |
| --- | --- | --- |
| Greedy Choice Property | Local best choice is part of global solution | Activity selection: earliest finish time |
| Optimal Substructure | Optimal solution built from optimal subproblems | MST: removing edge leaves smaller MST |
| No Overlapping Subproblems | Each choice independent of others | Coin change with standard denominations |

**Listing 13.1 — Greedy Coin Change (Standard Denominations)**

In [ ]:
def coin_change_greedy(coins, amount):
    """
    Greedy coin change - works for standard denominations.
    Always pick largest coin that fits.
    """
    coins = sorted(coins, reverse=True)  # Largest first
    result = []
    remaining = amount
    
    for coin in coins:
        while remaining >= coin:
            result.append(coin)
            remaining -= coin
    
    if remaining == 0:
        return result
    return None  # Can't make exact change

# Standard US coins - greedy works!
coins = [25, 10, 5, 1]
amount = 63
result = coin_change_greedy(coins, amount)
print(f"Amount: {amount} cents")
print(f"Coins used: {result}")
print(f"Number of coins: {len(result)}")

# Greedy FAILS for non-standard denominations
coins = [1, 3, 4]
amount = 6
greedy_result = coin_change_greedy(coins, amount)
print(f"\nNon-standard coins {coins}, amount {amount}:")
print(f"Greedy: {greedy_result} ({len(greedy_result)} coins)")
print(f"Optimal: [3, 3] (2 coins) - Greedy fails!")

***Figure 13.1:** Greedy works for standard coins but fails for arbitrary denominations.*

---
## 2. Greedy vs Dynamic Programming


**Listing 13.2 — Comparing Greedy and DP**

In [ ]:
# Problem: Maximize value with weight constraint

def fractional_knapsack(items, capacity):
    """
    Greedy works for FRACTIONAL knapsack.
    Take items by value/weight ratio.
    """
    # Sort by value per weight (descending)
    items = sorted(items, key=lambda x: x[1]/x[0], reverse=True)
    
    total_value = 0
    remaining = capacity
    
    for weight, value in items:
        if remaining >= weight:
            total_value += value
            remaining -= weight
        else:
            # Take fraction
            total_value += value * (remaining / weight)
            break
    
    return total_value

def knapsack_01_dp(items, capacity):
    """
    DP required for 0/1 knapsack.
    Can't take fractions of items.
    """
    n = len(items)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        weight, value = items[i - 1]
        for w in range(capacity + 1):
            dp[i][w] = dp[i - 1][w]
            if weight <= w:
                dp[i][w] = max(dp[i][w], dp[i - 1][w - weight] + value)
    
    return dp[n][capacity]

# Test: (weight, value) pairs
items = [(10, 60), (20, 100), (30, 120)]
capacity = 50

print(f"Items (weight, value): {items}")
print(f"Capacity: {capacity}")
print(f"\nFractional Knapsack (Greedy): {fractional_knapsack(items, capacity)}")
print(f"0/1 Knapsack (DP): {knapsack_01_dp(items, capacity)}")

***Figure 13.2:** Fractional knapsack allows greedy; 0/1 knapsack requires DP.*

**Listing 13.3 — When Greedy Fails: Optimal Path**

In [ ]:
# Greedy fails for finding longest path in DAG

def greedy_path(graph, start, end):
    """Greedy: always go to neighbor with highest edge weight."""
    path = [start]
    total = 0
    current = start
    
    while current != end:
        neighbors = graph.get(current, {})
        if not neighbors:
            return None, 0
        # Greedy: pick highest weight edge
        next_node = max(neighbors, key=neighbors.get)
        total += neighbors[next_node]
        path.append(next_node)
        current = next_node
    
    return path, total

def dp_longest_path(graph, start, end):
    """DP: consider all paths."""
    from functools import lru_cache
    
    @lru_cache(maxsize=None)
    def longest(node):
        if node == end:
            return 0, [end]
        
        best_dist = float('-inf')
        best_path = None
        
        for neighbor, weight in graph.get(node, {}).items():
            dist, path = longest(neighbor)
            if dist + weight > best_dist:
                best_dist = dist + weight
                best_path = [node] + path
        
        return best_dist, best_path
    
    return longest(start)

# Graph where greedy fails
graph = {
    'A': {'B': 1, 'C': 100},
    'B': {'D': 1000},
    'C': {'D': 1}
}

print("Graph: A->B(1), A->C(100), B->D(1000), C->D(1)")
print(f"\nGreedy path A→D: {greedy_path(graph, 'A', 'D')}")
print(f"Optimal path A→D: {dp_longest_path(graph, 'A', 'D')}")
print("\nGreedy chose C because 100 > 1, but missed B→D(1000)!")

***Figure 13.3:** Greedy's local choice (100 > 1) misses the globally optimal path.*

---
## 3. Activity Selection


> 📖 **Definition:** The **activity selection problem** asks: given activities with start and end times, select the maximum number of non-overlapping activities. The greedy approach is to always select the activity that finishes earliest.

**Listing 13.4 — Activity Selection (Maximum Activities)**

In [ ]:
def activity_selection(activities):
    """
    Select maximum non-overlapping activities.
    Greedy: always pick activity that ends earliest.
    Time: O(n log n) for sorting
    """
    # Sort by end time
    activities = sorted(activities, key=lambda x: x[1])
    
    selected = [activities[0]]
    last_end = activities[0][1]
    
    for start, end in activities[1:]:
        if start >= last_end:  # No overlap
            selected.append((start, end))
            last_end = end
    
    return selected

# Test
activities = [(1, 4), (3, 5), (0, 6), (5, 7), (3, 9), (5, 9), (6, 10), (8, 11), (8, 12), (2, 14), (12, 16)]
print("Activities (start, end):")
for i, act in enumerate(activities):
    print(f"  {i}: {act}")

selected = activity_selection(activities)
print(f"\nSelected activities: {selected}")
print(f"Maximum count: {len(selected)}")

***Figure 13.4:** Earliest finish time guarantees maximum activities.*

**Listing 13.5 — Why Earliest Finish Time Works**

In [ ]:
def visualize_activity_selection(activities):
    """Visualize the greedy choice."""
    activities = sorted(activities, key=lambda x: x[1])
    
    print("Timeline (sorted by end time):")
    print("-" * 50)
    
    selected = []
    last_end = -1
    
    for i, (start, end) in enumerate(activities):
        timeline = ['.'] * 20
        for t in range(start, min(end, 20)):
            timeline[t] = '#'
        
        status = ""
        if start >= last_end:
            selected.append((start, end))
            last_end = end
            status = " ✓ SELECTED"
        else:
            status = " ✗ overlaps"
        
        print(f"{i}: {''.join(timeline)} ({start}-{end}){status}")
    
    print("-" * 50)
    print(f"Selected {len(selected)} activities")

activities = [(0, 3), (1, 4), (3, 5), (4, 7), (5, 8), (7, 10)]
visualize_activity_selection(activities)

***Figure 13.5:** Each selection leaves maximum room for future activities.*

---
## 4. Interval Problems


**Listing 13.6 — Merge Intervals**

In [ ]:
def merge_intervals(intervals):
    """
    Merge overlapping intervals.
    Greedy: sort by start, extend or add new interval.
    """
    if not intervals:
        return []
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    merged = [intervals[0]]
    
    for start, end in intervals[1:]:
        if start <= merged[-1][1]:  # Overlapping
            merged[-1][1] = max(merged[-1][1], end)
        else:
            merged.append([start, end])
    
    return merged

# Test
intervals = [[1, 3], [2, 6], [8, 10], [15, 18]]
print(f"Original: {intervals}")
print(f"Merged:   {merge_intervals(intervals)}")

intervals = [[1, 4], [4, 5]]
print(f"\nOriginal: {intervals}")
print(f"Merged:   {merge_intervals(intervals)}")

***Figure 13.6:** Sort by start, greedily extend current interval or start new one.*

**Listing 13.7 — Non-Overlapping Intervals (Minimum Removals)**

In [ ]:
def erase_overlap_intervals(intervals):
    """
    Minimum intervals to remove for no overlap.
    Equivalent to: max non-overlapping = n - removals
    """
    if not intervals:
        return 0
    
    # Sort by end time (same as activity selection)
    intervals.sort(key=lambda x: x[1])
    
    count = 1  # Keep first interval
    last_end = intervals[0][1]
    
    for start, end in intervals[1:]:
        if start >= last_end:  # No overlap
            count += 1
            last_end = end
        # else: skip (remove) this interval
    
    return len(intervals) - count

# Test
intervals = [[1, 2], [2, 3], [3, 4], [1, 3]]
print(f"Intervals: {intervals}")
print(f"Remove {erase_overlap_intervals(intervals)} to eliminate overlaps")

intervals = [[1, 2], [1, 2], [1, 2]]
print(f"\nIntervals: {intervals}")
print(f"Remove {erase_overlap_intervals(intervals)} to eliminate overlaps")

***Figure 13.7:** Minimum removals = total - maximum non-overlapping.*

**Listing 13.8 — Meeting Rooms II (Minimum Rooms)**

In [ ]:
import heapq

def min_meeting_rooms(intervals):
    """
    Minimum conference rooms needed.
    Greedy with heap: track when rooms become free.
    """
    if not intervals:
        return 0
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    # Min-heap of end times (when rooms free up)
    rooms = []
    heapq.heappush(rooms, intervals[0][1])
    
    for start, end in intervals[1:]:
        # If earliest ending room is free
        if start >= rooms[0]:
            heapq.heappop(rooms)  # Reuse room
        
        heapq.heappush(rooms, end)
    
    return len(rooms)

# Alternative: sweep line algorithm
def min_meeting_rooms_sweep(intervals):
    """Sweep line: track concurrent meetings."""
    events = []
    for start, end in intervals:
        events.append((start, 1))   # Meeting starts
        events.append((end, -1))    # Meeting ends
    
    events.sort()
    
    max_rooms = current = 0
    for time, delta in events:
        current += delta
        max_rooms = max(max_rooms, current)
    
    return max_rooms

# Test
intervals = [[0, 30], [5, 10], [15, 20]]
print(f"Meetings: {intervals}")
print(f"Minimum rooms (heap): {min_meeting_rooms(intervals)}")
print(f"Minimum rooms (sweep): {min_meeting_rooms_sweep(intervals)}")

***Figure 13.8:** Heap tracks room availability; sweep line counts concurrent events.*

**Listing 13.9 — Insert Interval**

In [ ]:
def insert_interval(intervals, new_interval):
    """
    Insert new interval into sorted list, merge if needed.
    """
    result = []
    i = 0
    n = len(intervals)
    
    # Add all intervals that end before new one starts
    while i < n and intervals[i][1] < new_interval[0]:
        result.append(intervals[i])
        i += 1
    
    # Merge overlapping intervals
    while i < n and intervals[i][0] <= new_interval[1]:
        new_interval[0] = min(new_interval[0], intervals[i][0])
        new_interval[1] = max(new_interval[1], intervals[i][1])
        i += 1
    
    result.append(new_interval)
    
    # Add remaining intervals
    while i < n:
        result.append(intervals[i])
        i += 1
    
    return result

# Test
intervals = [[1, 3], [6, 9]]
new = [2, 5]
print(f"Intervals: {intervals}")
print(f"Insert: {new}")
print(f"Result: {insert_interval(intervals, new)}")

intervals = [[1, 2], [3, 5], [6, 7], [8, 10], [12, 16]]
new = [4, 8]
print(f"\nIntervals: {intervals}")
print(f"Insert: {new}")
print(f"Result: {insert_interval(intervals, new)}")

***Figure 13.9:** Three phases: add before, merge overlapping, add after.*

---
## 5. Fractional Knapsack


**Listing 13.10 — Fractional Knapsack**

In [ ]:
def fractional_knapsack_detailed(items, capacity):
    """
    Fractional knapsack with detailed output.
    items: list of (weight, value) tuples
    """
    # Calculate value per unit weight
    items_with_ratio = []
    for i, (w, v) in enumerate(items):
        items_with_ratio.append((v / w, w, v, i))
    
    # Sort by ratio descending
    items_with_ratio.sort(reverse=True)
    
    total_value = 0
    remaining = capacity
    taken = []
    
    for ratio, weight, value, idx in items_with_ratio:
        if remaining >= weight:
            # Take entire item
            total_value += value
            taken.append((idx, 1.0, value))
            remaining -= weight
        elif remaining > 0:
            # Take fraction
            fraction = remaining / weight
            gained = value * fraction
            total_value += gained
            taken.append((idx, fraction, gained))
            remaining = 0
            break
    
    return total_value, taken

# Test
items = [(10, 60), (20, 100), (30, 120)]  # (weight, value)
capacity = 50

print(f"Items (weight, value):")
for i, (w, v) in enumerate(items):
    print(f"  {i}: weight={w}, value={v}, ratio={v/w:.2f}")

print(f"\nCapacity: {capacity}")
total, taken = fractional_knapsack_detailed(items, capacity)

print(f"\nTaken:")
for idx, frac, val in taken:
    print(f"  Item {idx}: {frac*100:.0f}% -> value {val:.2f}")
print(f"\nTotal value: {total}")

***Figure 13.10:** Greedy by value/weight ratio gives optimal fractional solution.*

**Listing 13.29 — Fractional vs 0/1 Knapsack Comparison**

In [ ]:
# Compare fractional (greedy) vs 0/1 (DP) knapsack

def fractional_knapsack(items, capacity):
    """Greedy: sort by value/weight ratio."""
    items = sorted(items, key=lambda x: x[1]/x[0], reverse=True)
    total = 0
    for weight, value in items:
        if capacity >= weight:
            total += value
            capacity -= weight
        else:
            total += value * (capacity / weight)
            break
    return total

def knapsack_01(items, capacity):
    """DP: consider all subsets."""
    n = len(items)
    dp = [0] * (capacity + 1)
    for w, v in items:
        for c in range(capacity, w - 1, -1):
            dp[c] = max(dp[c], dp[c - w] + v)
    return dp[capacity]

items = [(10, 60), (20, 100), (30, 120)]
capacity = 50

print(f"Items (weight, value): {items}")
print(f"Capacity: {capacity}")
print(f"\nFractional (Greedy): {fractional_knapsack(items, capacity)}")
print(f"0/1 (DP): {knapsack_01(items, capacity)}")
print("\nFractional can take 10 + 20 + 20/30 of item 3 = 60 + 100 + 80 = 240")
print("0/1 takes items 2 and 3 = 100 + 120 = 220")

***Figure 13.29:** Fractional gets more value by taking partial items.*

---
## 6. Huffman Coding


> 📖 **Definition:** **Huffman coding** is a greedy algorithm for lossless data compression. It assigns shorter codes to more frequent characters, building an optimal prefix-free binary code. The greedy choice: always combine the two nodes with lowest frequency.

**Listing 13.11 — Huffman Tree Construction**

In [ ]:
import heapq
from collections import Counter

class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    """Build Huffman tree from text."""
    # Count frequencies
    freq = Counter(text)
    
    # Create leaf nodes and add to min-heap
    heap = [HuffmanNode(char, f) for char, f in freq.items()]
    heapq.heapify(heap)
    
    # Build tree by combining lowest frequency nodes
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        
        internal = HuffmanNode(None, left.freq + right.freq)
        internal.left = left
        internal.right = right
        
        heapq.heappush(heap, internal)
    
    return heap[0]

def get_codes(root, code="", codes=None):
    """Extract codes from Huffman tree."""
    if codes is None:
        codes = {}
    
    if root.char is not None:
        codes[root.char] = code if code else "0"
        return codes
    
    get_codes(root.left, code + "0", codes)
    get_codes(root.right, code + "1", codes)
    return codes

# Test
text = "abracadabra"
print(f"Text: {text}")
print(f"Frequencies: {dict(Counter(text))}")

tree = build_huffman_tree(text)
codes = get_codes(tree)
print(f"\nHuffman codes:")
for char, code in sorted(codes.items()):
    print(f"  '{char}': {code}")

encoded = ''.join(codes[c] for c in text)
print(f"\nEncoded: {encoded}")
print(f"Original bits: {len(text) * 8}")
print(f"Encoded bits: {len(encoded)}")

***Figure 13.11:** Frequent characters get shorter codes; 'a' appears most so gets shortest code.*

**Listing 13.12 — Huffman Encoding and Decoding**

In [ ]:
import heapq
from collections import Counter

class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def huffman_encode(text):
    """Encode text using Huffman coding."""
    if len(set(text)) == 1:
        return "0" * len(text), {text[0]: "0"}
    
    freq = Counter(text)
    heap = [HuffmanNode(c, f) for c, f in freq.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        left, right = heapq.heappop(heap), heapq.heappop(heap)
        internal = HuffmanNode(None, left.freq + right.freq)
        internal.left, internal.right = left, right
        heapq.heappush(heap, internal)
    
    codes = {}
    def build_codes(node, code=""):
        if node.char:
            codes[node.char] = code
        else:
            build_codes(node.left, code + "0")
            build_codes(node.right, code + "1")
    build_codes(heap[0])
    
    encoded = ''.join(codes[c] for c in text)
    return encoded, codes

def huffman_decode(encoded, codes):
    """Decode Huffman encoded string."""
    reverse_codes = {v: k for k, v in codes.items()}
    
    decoded = []
    current = ""
    for bit in encoded:
        current += bit
        if current in reverse_codes:
            decoded.append(reverse_codes[current])
            current = ""
    
    return ''.join(decoded)

# Test
text = "hello world"
encoded, codes = huffman_encode(text)
decoded = huffman_decode(encoded, codes)

print(f"Original: '{text}'")
print(f"Codes: {codes}")
print(f"Encoded: {encoded}")
print(f"Decoded: '{decoded}'")
print(f"Match: {text == decoded}")

***Figure 13.12:** Prefix-free codes allow unambiguous decoding.*

---
## 7. Minimum Spanning Trees


> 📖 **Definition:** A **Minimum Spanning Tree (MST)** connects all vertices in a weighted graph with minimum total edge weight. Two classic greedy algorithms: **Kruskal's** (sort edges, add if no cycle) and **Prim's** (grow tree from a vertex).

**Listing 13.13 — Kruskal's Algorithm with Union-Find**

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True

def kruskal_mst(n, edges):
    """
    Kruskal's MST algorithm.
    edges: list of (weight, u, v)
    Greedy: sort edges, add smallest that doesn't create cycle.
    Time: O(E log E)
    """
    edges.sort()  # Sort by weight
    uf = UnionFind(n)
    
    mst = []
    total_weight = 0
    
    for weight, u, v in edges:
        if uf.union(u, v):  # No cycle
            mst.append((u, v, weight))
            total_weight += weight
            if len(mst) == n - 1:
                break
    
    return mst, total_weight

# Test
n = 4
edges = [
    (1, 0, 1), (2, 1, 2), (3, 0, 2),
    (4, 2, 3), (5, 1, 3), (6, 0, 3)
]
print(f"Vertices: {n}")
print(f"Edges (weight, u, v): {edges}")

mst, total = kruskal_mst(n, edges)
print(f"\nMST edges: {mst}")
print(f"Total weight: {total}")

***Figure 13.13:** Kruskal's: greedily add smallest edge that doesn't create a cycle.*

**Listing 13.14 — Prim's Algorithm with Heap**

In [ ]:
import heapq

def prim_mst(n, adj):
    """
    Prim's MST algorithm.
    adj: adjacency list with (neighbor, weight)
    Greedy: always add closest vertex to current tree.
    Time: O(E log V) with heap
    """
    visited = [False] * n
    mst = []
    total_weight = 0
    
    # Start from vertex 0: (weight, vertex, parent)
    heap = [(0, 0, -1)]
    
    while heap and len(mst) < n:
        weight, u, parent = heapq.heappop(heap)
        
        if visited[u]:
            continue
        
        visited[u] = True
        if parent != -1:
            mst.append((parent, u, weight))
            total_weight += weight
        
        for v, w in adj[u]:
            if not visited[v]:
                heapq.heappush(heap, (w, v, u))
    
    return mst, total_weight

# Test - same graph as Kruskal's
adj = {
    0: [(1, 1), (2, 3), (3, 6)],
    1: [(0, 1), (2, 2), (3, 5)],
    2: [(0, 3), (1, 2), (3, 4)],
    3: [(0, 6), (1, 5), (2, 4)]
}

mst, total = prim_mst(4, adj)
print(f"Prim's MST edges (parent, vertex, weight): {mst}")
print(f"Total weight: {total}")

***Figure 13.14:** Prim's: grow tree by always adding the nearest unvisited vertex.*

---
## 8. Dijkstra's Shortest Path


**Listing 13.15 — Dijkstra's Algorithm**

In [ ]:
import heapq

def dijkstra(n, adj, start):
    """
    Dijkstra's shortest path from start to all vertices.
    Greedy: always process vertex with smallest known distance.
    Time: O((V + E) log V) with heap
    Only works with non-negative weights!
    """
    dist = [float('inf')] * n
    dist[start] = 0
    
    heap = [(0, start)]  # (distance, vertex)
    
    while heap:
        d, u = heapq.heappop(heap)
        
        if d > dist[u]:  # Already found better path
            continue
        
        for v, weight in adj[u]:
            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
                heapq.heappush(heap, (dist[v], v))
    
    return dist

# Test
adj = {
    0: [(1, 4), (2, 1)],
    1: [(3, 1)],
    2: [(1, 2), (3, 5)],
    3: []
}

dist = dijkstra(4, adj, 0)
print("Graph:")
print("  0 --4--> 1 --1--> 3")
print("  |        ^        ^")
print("  1        2        5")
print("  v        |        |")
print("  2 -------+--------+")

print(f"\nShortest distances from 0:")
for v, d in enumerate(dist):
    print(f"  0 -> {v}: {d}")

***Figure 13.15:** Dijkstra greedily processes vertices in order of distance.*

**Listing 13.16 — Dijkstra with Path Reconstruction**

In [ ]:
import heapq

def dijkstra_with_path(n, adj, start, end):
    """Dijkstra's with path reconstruction."""
    dist = [float('inf')] * n
    dist[start] = 0
    parent = [-1] * n
    
    heap = [(0, start)]
    
    while heap:
        d, u = heapq.heappop(heap)
        
        if u == end:
            break
        
        if d > dist[u]:
            continue
        
        for v, weight in adj[u]:
            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
                parent[v] = u
                heapq.heappush(heap, (dist[v], v))
    
    # Reconstruct path
    if dist[end] == float('inf'):
        return None, float('inf')
    
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = parent[current]
    
    return path[::-1], dist[end]

# Test
adj = {
    0: [(1, 4), (2, 1)],
    1: [(3, 1)],
    2: [(1, 2), (3, 5)],
    3: []
}

path, dist = dijkstra_with_path(4, adj, 0, 3)
print(f"Shortest path 0 -> 3: {path}")
print(f"Distance: {dist}")

***Figure 13.16:** Track parent pointers to reconstruct the actual shortest path.*

---
## 9. Task Scheduling


**Listing 13.17 — Job Scheduling with Deadlines**

In [ ]:
def job_sequencing(jobs):
    """
    Maximize profit by scheduling jobs before their deadlines.
    Each job takes 1 time unit.
    jobs: list of (job_id, deadline, profit)
    """
    # Sort by profit descending
    jobs = sorted(jobs, key=lambda x: x[2], reverse=True)
    
    max_deadline = max(job[1] for job in jobs)
    slots = [None] * max_deadline
    
    total_profit = 0
    scheduled = []
    
    for job_id, deadline, profit in jobs:
        # Find latest available slot before deadline
        for slot in range(min(deadline, max_deadline) - 1, -1, -1):
            if slots[slot] is None:
                slots[slot] = job_id
                total_profit += profit
                scheduled.append((job_id, slot + 1, profit))
                break
    
    return scheduled, total_profit

# Test
jobs = [
    ('a', 2, 100),
    ('b', 1, 19),
    ('c', 2, 27),
    ('d', 1, 25),
    ('e', 3, 15)
]

print("Jobs (id, deadline, profit):")
for job in jobs:
    print(f"  {job}")

scheduled, profit = job_sequencing(jobs)
print(f"\nScheduled jobs:")
for job_id, slot, p in scheduled:
    print(f"  Job '{job_id}' in slot {slot}, profit {p}")
print(f"\nTotal profit: {profit}")

***Figure 13.17:** Schedule highest profit jobs first, as late as possible before deadline.*

**Listing 13.18 — Minimum Platforms (Train Station)**

In [ ]:
def min_platforms(arrivals, departures):
    """
    Minimum platforms needed at a train station.
    Similar to meeting rooms problem.
    """
    events = []
    for arr in arrivals:
        events.append((arr, 1))   # Arrival
    for dep in departures:
        events.append((dep, -1))  # Departure
    
    events.sort()
    
    max_platforms = current = 0
    for time, delta in events:
        current += delta
        max_platforms = max(max_platforms, current)
    
    return max_platforms

# Test
arrivals = [900, 940, 950, 1100, 1500, 1800]
departures = [910, 1200, 1120, 1130, 1900, 2000]

print(f"Train arrivals:   {arrivals}")
print(f"Train departures: {departures}")
print(f"Minimum platforms needed: {min_platforms(arrivals, departures)}")

***Figure 13.18:** Sweep line tracks concurrent trains at any moment.*

**Listing 13.19 — Task Scheduler with Cooldown**

In [ ]:
from collections import Counter

def least_interval(tasks, n):
    """
    Minimum time to complete all tasks with cooldown n between same tasks.
    Greedy: always schedule most frequent task.
    """
    freq = Counter(tasks)
    max_freq = max(freq.values())
    
    # Count tasks with max frequency
    max_count = sum(1 for f in freq.values() if f == max_freq)
    
    # Formula: (max_freq - 1) * (n + 1) + max_count
    # This is the minimum slots needed for the most frequent tasks
    formula_result = (max_freq - 1) * (n + 1) + max_count
    
    # But can't be less than total tasks
    return max(formula_result, len(tasks))

# Test
tasks = ['A', 'A', 'A', 'B', 'B', 'B']
n = 2
print(f"Tasks: {tasks}")
print(f"Cooldown: {n}")
print(f"Minimum intervals: {least_interval(tasks, n)}")
# A -> B -> idle -> A -> B -> idle -> A -> B

tasks = ['A', 'A', 'A', 'B', 'B', 'B']
n = 0
print(f"\nTasks: {tasks}")
print(f"Cooldown: {n}")
print(f"Minimum intervals: {least_interval(tasks, n)}")

***Figure 13.19:** Most frequent task determines minimum time; fill idle slots with others.*

---
## 10. Common Patterns


### Pattern 1: Jump Game

**Listing 13.20 — Jump Game**

In [ ]:
def can_jump(nums):
    """
    Can we reach the last index?
    Greedy: track farthest reachable position.
    """
    max_reach = 0
    
    for i, jump in enumerate(nums):
        if i > max_reach:
            return False
        max_reach = max(max_reach, i + jump)
        if max_reach >= len(nums) - 1:
            return True
    
    return True

def min_jumps(nums):
    """
    Minimum jumps to reach last index.
    Greedy: BFS-like level traversal.
    """
    if len(nums) <= 1:
        return 0
    
    jumps = 0
    current_end = 0
    farthest = 0
    
    for i in range(len(nums) - 1):
        farthest = max(farthest, i + nums[i])
        
        if i == current_end:
            jumps += 1
            current_end = farthest
            
            if current_end >= len(nums) - 1:
                break
    
    return jumps

# Test
nums = [2, 3, 1, 1, 4]
print(f"Array: {nums}")
print(f"Can reach end: {can_jump(nums)}")
print(f"Minimum jumps: {min_jumps(nums)}")

nums = [3, 2, 1, 0, 4]
print(f"\nArray: {nums}")
print(f"Can reach end: {can_jump(nums)}")

***Figure 13.20:** Track reachable range; expand greedily when hitting boundary.*

### Pattern 2: Gas Station

**Listing 13.21 — Gas Station Circuit**

In [ ]:
def can_complete_circuit(gas, cost):
    """
    Find starting station to complete circuit.
    Greedy: if we can't reach station i from start,
    start must be after i.
    """
    total_tank = 0
    current_tank = 0
    start = 0
    
    for i in range(len(gas)):
        diff = gas[i] - cost[i]
        total_tank += diff
        current_tank += diff
        
        if current_tank < 0:
            # Can't reach station i+1 from start
            # New start must be after i
            start = i + 1
            current_tank = 0
    
    return start if total_tank >= 0 else -1

# Test
gas = [1, 2, 3, 4, 5]
cost = [3, 4, 5, 1, 2]
print(f"Gas:  {gas}")
print(f"Cost: {cost}")
print(f"Start from station: {can_complete_circuit(gas, cost)}")

gas = [2, 3, 4]
cost = [3, 4, 3]
print(f"\nGas:  {gas}")
print(f"Cost: {cost}")
print(f"Start from station: {can_complete_circuit(gas, cost)}")

***Figure 13.21:** If total gas >= total cost, solution exists; find valid starting point.*

### Pattern 3: Candy Distribution

**Listing 13.22 — Candy Distribution**

In [ ]:
def candy(ratings):
    """
    Distribute candies: higher rating gets more than neighbors.
    Two-pass greedy: left to right, then right to left.
    """
    n = len(ratings)
    candies = [1] * n
    
    # Left to right: handle ascending sequences
    for i in range(1, n):
        if ratings[i] > ratings[i - 1]:
            candies[i] = candies[i - 1] + 1
    
    # Right to left: handle descending sequences
    for i in range(n - 2, -1, -1):
        if ratings[i] > ratings[i + 1]:
            candies[i] = max(candies[i], candies[i + 1] + 1)
    
    return sum(candies), candies

# Test
ratings = [1, 0, 2]
total, dist = candy(ratings)
print(f"Ratings: {ratings}")
print(f"Candies: {dist}")
print(f"Total: {total}")

ratings = [1, 2, 2]
total, dist = candy(ratings)
print(f"\nRatings: {ratings}")
print(f"Candies: {dist}")
print(f"Total: {total}")

***Figure 13.22:** Two passes handle both increasing and decreasing sequences.*

### Pattern 4: Partition Labels

**Listing 13.23 — Partition Labels**

In [ ]:
def partition_labels(s):
    """
    Partition string so each letter appears in at most one part.
    Greedy: extend partition until all letters in it have ended.
    """
    # Find last occurrence of each character
    last = {c: i for i, c in enumerate(s)}
    
    partitions = []
    start = end = 0
    
    for i, c in enumerate(s):
        end = max(end, last[c])  # Extend if needed
        
        if i == end:  # All chars in partition have ended
            partitions.append(end - start + 1)
            start = i + 1
    
    return partitions

# Test
s = "ababcbacadefegdehijhklij"
print(f"String: {s}")
result = partition_labels(s)
print(f"Partition sizes: {result}")

# Show partitions
idx = 0
parts = []
for size in result:
    parts.append(s[idx:idx + size])
    idx += size
print(f"Partitions: {parts}")

***Figure 13.23:** Track last occurrence; partition when current position equals max end.*

### Pattern 5: Best Time to Buy and Sell Stock II

**Listing 13.30 — Stock Profit (Multiple Transactions)**

In [ ]:
def max_profit_multiple(prices):
    """
    Maximum profit with unlimited buy/sell transactions.
    Greedy: collect every upward price movement.
    """
    profit = 0
    for i in range(1, len(prices)):
        if prices[i] > prices[i - 1]:
            profit += prices[i] - prices[i - 1]
    return profit

# Test
prices = [7, 1, 5, 3, 6, 4]
print(f"Prices: {prices}")
print(f"Max profit: {max_profit_multiple(prices)}")
# Buy at 1, sell at 5 (profit 4)
# Buy at 3, sell at 6 (profit 3)
# Total: 7

prices = [1, 2, 3, 4, 5]
print(f"\nPrices: {prices}")
print(f"Max profit: {max_profit_multiple(prices)}")
# Buy at 1, sell at 5 = profit 4 (or collect each +1)

***Figure 13.30:** Greedy collects every positive price difference.*

### Pattern 6: Remove K Digits

**Listing 13.31 — Remove K Digits for Smallest Number**

In [ ]:
def remove_k_digits(num, k):
    """
    Remove k digits to get smallest possible number.
    Greedy: remove digit if next digit is smaller (using stack).
    """
    stack = []
    
    for digit in num:
        while k > 0 and stack and stack[-1] > digit:
            stack.pop()
            k -= 1
        stack.append(digit)
    
    # Remove remaining k digits from end
    stack = stack[:-k] if k else stack
    
    # Remove leading zeros
    result = ''.join(stack).lstrip('0')
    return result or '0'

# Test
num = "1432219"
k = 3
print(f"Number: {num}, remove {k} digits")
print(f"Result: {remove_k_digits(num, k)}")  # 1219

num = "10200"
k = 1
print(f"\nNumber: {num}, remove {k} digit")
print(f"Result: {remove_k_digits(num, k)}")  # 200

***Figure 13.31:** Monotonic stack removes larger digits when smaller digit follows.*

### Pattern 7: Maximum Number of Events

**Listing 13.32 — Maximum Events Attended**

In [ ]:
import heapq

def max_events(events):
    """
    Maximum events you can attend (one event per day).
    Greedy: each day, attend event ending soonest.
    """
    events = sorted(events)
    heap = []  # End days of available events
    day = 0
    count = 0
    i = 0
    n = len(events)
    
    while i < n or heap:
        if not heap:
            day = events[i][0]
        
        # Add all events starting today or earlier
        while i < n and events[i][0] <= day:
            heapq.heappush(heap, events[i][1])
            i += 1
        
        # Remove expired events
        while heap and heap[0] < day:
            heapq.heappop(heap)
        
        # Attend event ending soonest
        if heap:
            heapq.heappop(heap)
            count += 1
        
        day += 1
    
    return count

# Test
events = [[1, 2], [2, 3], [3, 4]]
print(f"Events: {events}")
print(f"Max events: {max_events(events)}")  # 3

events = [[1, 2], [2, 3], [3, 4], [1, 2]]
print(f"\nEvents: {events}")
print(f"Max events: {max_events(events)}")  # 4

***Figure 13.32:** Use heap to always attend event ending soonest.*

### Pattern 8: Reorganize String

**Listing 13.33 — Reorganize String (No Adjacent Same)**

In [ ]:
import heapq
from collections import Counter

def reorganize_string(s):
    """
    Rearrange so no two adjacent chars are same.
    Greedy: always place most frequent char not just used.
    """
    freq = Counter(s)
    
    # Check if possible
    max_freq = max(freq.values())
    if max_freq > (len(s) + 1) // 2:
        return ""
    
    # Max heap of (-count, char)
    heap = [(-count, char) for char, count in freq.items()]
    heapq.heapify(heap)
    
    result = []
    prev_count, prev_char = 0, ''
    
    while heap:
        count, char = heapq.heappop(heap)
        result.append(char)
        
        # Put previous char back if it has remaining count
        if prev_count < 0:
            heapq.heappush(heap, (prev_count, prev_char))
        
        prev_count, prev_char = count + 1, char
    
    return ''.join(result)

# Test
s = "aab"
print(f"String: {s}")
print(f"Reorganized: {reorganize_string(s)}")

s = "aaab"
print(f"\nString: {s}")
print(f"Reorganized: '{reorganize_string(s)}' (impossible)")

***Figure 13.33:** Alternate between most frequent characters using heap.*

- **Greedy choice property:** Local optimal leads to global optimal
- **Activity selection:** Sort by end time, pick non-overlapping
- **Interval problems:** Sort by start or end depending on goal
- **MST:** Kruskal's (edges) or Prim's (vertices)
- **Shortest path:** Dijkstra's for non-negative weights
- **Proof technique:** Exchange argument shows greedy is optimal

---
## 11. Common Pitfalls


### Pitfall 1: Greedy Doesn't Always Work

**Listing 13.24 — When Greedy Fails**

In [ ]:
# 0/1 Knapsack: Greedy by ratio fails

def greedy_knapsack_01(items, capacity):
    """WRONG: Greedy approach for 0/1 knapsack."""
    items = sorted(items, key=lambda x: x[1]/x[0], reverse=True)
    
    total = 0
    remaining = capacity
    for weight, value in items:
        if weight <= remaining:
            total += value
            remaining -= weight
    return total

def dp_knapsack_01(items, capacity):
    """CORRECT: DP approach."""
    n = len(items)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        w, v = items[i - 1]
        for c in range(capacity + 1):
            dp[i][c] = dp[i - 1][c]
            if w <= c:
                dp[i][c] = max(dp[i][c], dp[i - 1][c - w] + v)
    
    return dp[n][capacity]

# Counter-example where greedy fails
items = [(10, 60), (20, 100), (30, 120)]  # (weight, value)
capacity = 50

print("Items (weight, value):")
for w, v in items:
    print(f"  weight={w}, value={v}, ratio={v/w:.2f}")

print(f"\nCapacity: {capacity}")
print(f"Greedy result: {greedy_knapsack_01(items, capacity)}")
print(f"DP result: {dp_knapsack_01(items, capacity)}")
print("\nGreedy takes items 0+1 (value 160)")
print("Optimal takes items 1+2 (value 220)")

***Figure 13.24:** 0/1 knapsack requires considering combinations—greedy misses better options.*

### Pitfall 2: Wrong Sorting Criteria

**Listing 13.25 — Correct Sorting Matters**

In [ ]:
# Activity selection: which sorting is correct?

activities = [(1, 4), (3, 5), (0, 6), (5, 7), (3, 9), (5, 9), (6, 10)]

def by_start_time(activities):
    """WRONG: Sort by start time."""
    activities = sorted(activities, key=lambda x: x[0])
    selected = [activities[0]]
    for start, end in activities[1:]:
        if start >= selected[-1][1]:
            selected.append((start, end))
    return selected

def by_end_time(activities):
    """CORRECT: Sort by end time."""
    activities = sorted(activities, key=lambda x: x[1])
    selected = [activities[0]]
    for start, end in activities[1:]:
        if start >= selected[-1][1]:
            selected.append((start, end))
    return selected

def by_duration(activities):
    """WRONG: Sort by duration."""
    activities = sorted(activities, key=lambda x: x[1] - x[0])
    selected = [activities[0]]
    for start, end in activities[1:]:
        if start >= selected[-1][1]:
            selected.append((start, end))
    return selected

print(f"Activities: {activities}")
print(f"\nBy start time: {len(by_start_time(activities))} activities")
print(f"By end time:   {len(by_end_time(activities))} activities (CORRECT)")
print(f"By duration:   {len(by_duration(activities))} activities")

***Figure 13.25:** Different sorting criteria give different results—only end time is optimal.*

### Pitfall 3: Forgetting Edge Cases

**Listing 13.26 — Handle Edge Cases**

In [ ]:
def jump_game_robust(nums):
    """Jump game with edge case handling."""
    if not nums:
        return False
    if len(nums) == 1:
        return True  # Already at end
    
    max_reach = 0
    for i in range(len(nums)):
        if i > max_reach:
            return False
        max_reach = max(max_reach, i + nums[i])
        if max_reach >= len(nums) - 1:
            return True
    
    return False

# Edge cases
test_cases = [
    [],           # Empty
    [0],          # Single element, zero jump
    [1],          # Single element, can jump
    [0, 1],       # Stuck at start
    [1, 0, 1],    # Middle zero
]

for nums in test_cases:
    print(f"nums = {nums}: {jump_game_robust(nums)}")

***Figure 13.26:** Always handle empty, single element, and boundary cases.*

### Pitfall 4: Negative Weights in Dijkstra

**Listing 13.27 — Dijkstra Fails with Negative Weights**

In [ ]:
import heapq

def dijkstra(adj, start, n):
    dist = [float('inf')] * n
    dist[start] = 0
    heap = [(0, start)]
    
    while heap:
        d, u = heapq.heappop(heap)
        if d > dist[u]:
            continue
        for v, w in adj.get(u, []):
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                heapq.heappush(heap, (dist[v], v))
    return dist

# Graph with negative edge
adj = {
    0: [(1, 1), (2, 5)],
    1: [(2, -3)],  # Negative edge!
    2: []
}

print("Graph with negative edge:")
print("  0 --1--> 1 ---(-3)---> 2")
print("  0 --5--> 2")
print(f"\nDijkstra distances: {dijkstra(adj, 0, 3)}")
print("Correct: 0->1->2 = 1 + (-3) = -2")
print("\nDijkstra may give wrong answer with negative weights!")
print("Use Bellman-Ford instead for negative weights.")

***Figure 13.27:** Dijkstra's greedy assumption breaks with negative edges.*

### Pitfall 5: Not Proving Greedy Works

**Listing 13.28 — Exchange Argument Proof**

In [ ]:
"""
Exchange Argument for Activity Selection:

Claim: Always picking earliest-finish activity is optimal.

Proof:
1. Let G = greedy solution, O = any optimal solution
2. Let g1 be first activity in G (earliest finish)
3. Let o1 be first activity in O

Case: g1 finishes no later than o1
- Replace o1 with g1 in O
- Still valid (g1 finishes earlier, so same or more room for remaining)
- Still optimal (same number of activities)
- Now G and O have same first activity

4. Repeat argument for remaining activities
5. Eventually, G = O

This is called the "exchange argument" - we can exchange
any optimal solution to match greedy without losing optimality.
"""

print("Exchange Argument proves greedy is optimal:")
print()
print("1. Suppose greedy picks activity A that ends at time t")
print("2. Any other first choice B ends at time >= t")
print("3. Replacing B with A leaves MORE room for remaining activities")
print("4. So greedy choice is NEVER worse than any alternative")
print("5. Therefore greedy is optimal")
print()
print("ALWAYS prove greedy works before using it!")

***Figure 13.28:** Exchange argument: show greedy can replace any optimal solution.*

---
# 📝 Exercises


### Exercise 1: Maximum Units on a Truck  (⭐ Easy)

You can load boxes onto a truck with capacity truckSize. Given boxTypes where boxTypes[i] = [numberOfBoxes, unitsPerBox], return maximum units you can load.

**Expected:** (Expected: [[1,3],[2,2],[3,1]], truckSize=4 → 8)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Greedy Choice:**

                        Load boxes with most units per box first (higher value per space).
- **Hint 2 - Sort:**

                        Sort by unitsPerBox in descending order.
- **Hint 3 - Load:**

                        Take min(available boxes, remaining capacity) of each type.
</details>

In [ ]:
# ✏️ [EX1]
# Maximum Units on a Truck - Greedy

def maximum_units(box_types, truck_size):
    # Your code here
    pass

# Test your implementation (uncomment)
# box_types = [[1,3],[2,2],[3,1]]
# print(maximum_units(box_types, 4))  # 8

### Exercise 2: Boats to Save People  (⭐⭐ Medium)

Given people's weights and boat limit (max 2 people per boat), find minimum boats needed.

**Expected:** (Expected: [3,2,2,1], limit=3 → 3 boats)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sort:**

                        Sort people by weight to enable optimal pairing.
- **Hint 2 - Two Pointers:**

                        Try to pair lightest (left) with heaviest (right).
- **Hint 3 - Logic:**

                        If they fit together, both go. Otherwise heaviest goes alone.
</details>

In [ ]:
# ✏️ [EX2]
# Boats to Save People - Two Pointers

def num_rescue_boats(people, limit):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(num_rescue_boats([3, 2, 2, 1], 3))  # 3
# print(num_rescue_boats([3, 5, 3, 4], 5))  # 4

### Exercise 3: Assign Cookies  (⭐ Easy)

Given greed factors g[i] and cookie sizes s[j], assign cookies to maximize content children. A child is content if cookie size >= greed factor.

**Expected:** (Expected: g=[1,2,3], s=[1,1] → 1 child content)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sort Both:**

                        Sort children by greed and cookies by size.
- **Hint 2 - Greedy Match:**

                        Give smallest sufficient cookie to least greedy child.
- **Hint 3 - Two Pointers:**

                        If cookie satisfies child, move both. Else, try next cookie.
</details>

In [ ]:
# ✏️ [EX3]
# Assign Cookies - Greedy Matching

def find_content_children(g, s):
    # Your code here
    pass

# Test your implementation (uncomment)
# print(find_content_children([1, 2, 3], [1, 1]))  # 1
# print(find_content_children([1, 2], [1, 2, 3]))  # 2

### Exercise 4: Queue Reconstruction by Height  (⭐⭐⭐ Hard)

People described by [h, k] where h is height and k is number of taller people in front. Reconstruct the queue.

**Expected:** (Expected: Reconstruct based on height and k value)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sort Order:**

                        Sort by height descending, then by k ascending.
- **Hint 2 - Insertion Logic:**

                        Tallest first don't see shorter people, so insert at index k.
- **Hint 3 - Build Queue:**

                        Insert each person at position k in the result list.
</details>

In [ ]:
# ✏️ [EX4]
# Queue Reconstruction by Height

def reconstruct_queue(people):
    # Your code here
    pass

# Test your implementation (uncomment)
# people = [[7,0],[4,4],[7,1],[5,0],[6,1],[5,2]]
# print(reconstruct_queue(people))
# [[5,0],[7,0],[5,2],[6,1],[4,4],[7,1]]

### Exercise 5: Minimum Number of Arrows  (⭐⭐⭐ Hard)

Given balloons as [start, end] intervals on x-axis, find minimum arrows needed to burst all balloons. One arrow at x bursts all balloons where start ≤ x ≤ end.

**Expected:** (Expected: [[10,16],[2,8],[1,6],[7,12]] → 2 arrows)

<details>
<summary>💡 Hints</summary>

- **Hint 1 - Sort by End:**

                        Sort balloons by end point (like activity selection).
- **Hint 2 - Shoot Strategy:**

                        Shoot arrow at end of first balloon to maximize overlap.
- **Hint 3 - Skip Overlapping:**

                        Skip all balloons whose start <= current arrow position.
</details>

In [ ]:
# ✏️ [EX5]
# Minimum Number of Arrows - Interval Greedy

def find_min_arrow_shots(points):
    # Your code here
    pass

# Test your implementation (uncomment)
# points = [[10,16],[2,8],[1,6],[7,12]]
# print(find_min_arrow_shots(points))  # 2

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request
import time as _time, datetime as _dt

WEEK = "Week_13"
URL  = "https://script.google.com/macros/s/AKfycbxepk2NvNg3Whad-WOPxdZI-mWnVJeNKCsZVspvk7Ku5YHC_oWv7376VrWLn_30nyI_vw/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': len(_code) > 5
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': len(_code) > 5
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': len(_code) > 5
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'dsa-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
